In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [115]:
# --- MODEL GELİŞTİRME ---
""" Bir parçanın YouTube’da viral olup olmayacağını tahmin eden bir model geliştiriyorum."
    Hedef Değişken: Binary (0/1) → Parça viral mi değil mi?
    Viral Tanımı: Views > 10 milyon ise 1, değilse 0.
    Kullanacağımız Bağımsız değişkenler: Likes, Comments, StreamDanceability, Energy, Valence ve Kategorik Değişkenler (Albüm Tipi, Sanatçı Popülerliği...)
"""

' Bir parçanın YouTube’da viral olup olmayacağını tahmin eden bir model geliştiriyorum."\n    Hedef Değişken: Binary (0/1) → Parça viral mi değil mi?\n    Viral Tanımı: Views > 10 milyon ise 1, değilse 0.\n    Kullanacağımız Bağımsız değişkenler: Likes, Comments, StreamDanceability, Energy, Valence ve Kategorik Değişkenler (Albüm Tipi, Sanatçı Popülerliği...)\n'

In [116]:
df = pd.read_csv("Spotify Youtube Dataset.csv")

# Sütun Adlarını Sadeleştir: Bazı sütunlar fazla boşluk içerebilir veya karmaşık isimlere sahip olabilir. 
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df.columns


Index(['unnamed:_0', 'artist', 'url_spotify', 'track', 'album', 'album_type',
       'uri', 'danceability', 'energy', 'key', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'url_youtube', 'title', 'channel', 'views', 'likes',
       'comments', 'description', 'licensed', 'official_video', 'stream'],
      dtype='object')

In [117]:
# Kategorik Tür Dönüşümü
cat_cols = ['album_type', 'channel', 'artist', 'album']
for col in cat_cols:
    df[col] = df[col].astype('category')


In [118]:
df.isnull().sum()

unnamed:_0            0
artist                0
url_spotify           0
track                 0
album                 0
album_type            0
uri                   0
danceability          2
energy                2
key                   2
loudness              2
speechiness           2
acousticness          2
instrumentalness      2
liveness              2
valence               2
tempo                 2
duration_ms           2
url_youtube         470
title               470
channel             470
views               470
likes               541
comments            569
description         876
licensed            470
official_video      470
stream              576
dtype: int64

In [119]:
# Küçük eksikleri olan sayısal sütunlar için satırları silelim.
numeric_subset = ['danceability', 'energy', 'key', 'loudness', 'speechiness',
                  'acousticness', 'instrumentalness', 'liveness',
                  'valence', 'tempo', 'duration_ms']
df.dropna(subset = numeric_subset, inplace = True)

# YouTube ile ilgili sütunlarda eksik varsa, sil.
df.dropna(subset = ['url_youtube', 'title', 'channel', 'views'], inplace = True)

# Mantıksal sütunları bool tipine çevir ve eksikleri 'False' yap.
df['licensed'] = df['licensed'].map({'True': True, 'False': False})
df['official_video'] = df['official_video'].map({'True': True, 'False': False})
df['licensed'].fillna(False, inplace = True)
df['official_video'].fillna(False, inplace = True)

# Likes, Comments, Stream, ortanca ile doldur.
df['likes'].fillna(df['likes'].median(), inplace = True)
df['comments'].fillna(df['comments'].median(), inplace = True)
df['stream'].fillna(df['stream'].median(), inplace = True)

# İstenmeyen Sütunları Silme
df.drop(columns = ['unnamed:_0', 'uri', 'url_spotify', 'url_youtube', 'description', 'title', 'track'], inplace = True)


In [120]:
df.isnull().sum()

artist              0
album               0
album_type          0
danceability        0
energy              0
key                 0
loudness            0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
channel             0
views               0
likes               0
comments            0
licensed            0
official_video      0
stream              0
dtype: int64

In [121]:
df.columns

Index(['artist', 'album', 'album_type', 'danceability', 'energy', 'key',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'channel', 'views',
       'likes', 'comments', 'licensed', 'official_video', 'stream'],
      dtype='object')

In [132]:
# Views > 10 000 000 olanları viral = 1, diğerlerini 0 yap
df['viral'] = (df['views'] > 10_000_000).astype(int)

In [133]:
# Feature Engineering
# Like-to-View Oranı
df['like_view_ratio'] = df['likes'] / df['views']

# Comment-to-View Oranı
df['comment_view_ratio'] = df['comments'] / df['views']

# Bazı durumlarda izlenme sayısı sıfır olabilir. Sıfıra bölme hatasının önüne geçelim.
df['like_view_ratio'].fillna(0, inplace = True)
df['comment_view_ratio'].fillna(0, inplace = True)


In [134]:
df_encoded = pd.get_dummies(df, columns = cat_cols, drop_first = True)


In [135]:
cat_cols = ['album_type', 'channel', 'artist', 'album']
df_encoded = pd.get_dummies(df, columns = cat_cols, drop_first = True)


In [136]:
scaler = StandardScaler()

num_cols = ['views', 'likes', 'comments', 'like_view_ratio', 'comment_view_ratio',
            'danceability', 'energy', 'key', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence',
            'tempo', 'duration_ms']

df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])

In [137]:
X = df_encoded.drop('viral', axis = 1)  # Özellikler
y = df_encoded['viral'] # Hedef Değişken

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)


In [138]:
print(X.dtypes[X.dtypes == 'object'])


Series([], dtype: object)


In [139]:
rf = RandomForestClassifier(random_state = 42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [140]:
y_pred = rf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[2709    0]
 [   3 3363]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2709
           1       1.00      1.00      1.00      3366

    accuracy                           1.00      6075
   macro avg       1.00      1.00      1.00      6075
weighted avg       1.00      1.00      1.00      6075

